In [1]:
import requests
from lxml import etree
import threading
import time
import csv
import re

In [2]:
def check_value(title):
    pipe = ["維基","Server Error","Access denied","Yahoo is now","Page Not Found","痞客邦","百度","404","Security"]
    if any(x in title for x in pipe):
        return True
    elif (re.search("[^a-zA-Z0-9\b]+",title)):
        return False
    else:
        return True

In [3]:
def clean_news(l,all_ls):
    newslink = hostName+ l
    response = requests.get(newslink);
    html = etree.HTML(response.content)
    origin_news = "".join(html.xpath("//div[@class='jsx-1919529045 message']/text()"))
    if(len(origin_news)>70 or len(origin_news)<3 or check_value(origin_news)) :
        return 0;
    new_titles = html.xpath("//section[@class='jsx-588669885 links']//article[@class='jsx-1682249194 link']/h1/@title")
    reason = "".join(html.xpath("//section[@class='jsx-3677418999 jsx-423907629 section']//div/text()"))
    for i in new_titles:
        if(check_value(i)):
            return 0
        dictionary = {}
        dictionary["link"] = newslink
        dictionary["news1"] = origin_news
        dictionary["news2"] = i
        dictionary["fact"] = False
        dictionary["reason"] = reason
        all_ls.append(dictionary)

In [ ]:
hostName = "https://cofacts.g0v.tw"
response = requests.get("https://cofacts.g0v.tw/replies?after=&before=&filter=RUMOR")
html = etree.HTML(response.content)
newslink_ls = html.xpath('//a[@class="jsx-3552945388 jsx-3720007368 item"]/@href')
next_link = html.xpath('//a[@class="jsx-684928770"]/@href')[1]
total_number = html.xpath('//p[@class="jsx-3107014753"]/text()')[0]
all_ls = []
count = 0
while(True):
    threads = []
    for l in newslink_ls:
        count += 1
        threads.append(threading.Thread(target=clean_news,args=(l,all_ls)))
        threads[-1].start()
        if(count%100 == 0  and count != 0):
            print(count)
    
    for t in threads:
        t.join()

    response = requests.get("https://cofacts.g0v.tw/replies?"+next_link)
    html = etree.HTML(response.content)
    try:
        newslink_ls = html.xpath('//a[@class="jsx-3552945388 jsx-3720007368 item"]/@href')
        next_link = html.xpath('//a[@class="jsx-684928770"]/@href')[1]
    except:
        break
print("done")

In [ ]:
keys = all_ls[0].keys()
with open('fake.csv', 'w',encoding="utf8") as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(all_ls)

****

In [11]:
from gensim.models import word2vec
from gensim import corpora, models, similarities
from collections import defaultdict
from langconv import *
import numpy as np
import jieba
import re
import csv

In [2]:
corpora_dict_path = 'corpora_dict.dict'
tfidf_model_path = "./tfidf.model"
corpus_path = './corpus.mm'
corpus = corpora.MmCorpus(corpus_path)
tfidf = models.TfidfModel.load(tfidf_model_path)
corpus_tfidf = tfidf[corpus]
dictionary = corpora.Dictionary.load(corpora_dict_path)
index = similarities.Similarity("./tempName",corpus_tfidf,len(dictionary.token2id))

In [3]:
def clean_sentence(sentence):
    r = '[’!"#$%&\'()*+,./:;<=>?@，。?★、…【】\n《》\s？“”‘’！[\\]^_`{|}~「」:]+'
    sentence = Converter('zh-hant').convert(sentence)
    words = re.sub(r,"",sentence)
    wordarr = [word for word in jieba.cut(words)]
    return " ".join(wordarr)

In [4]:
def calRelateRate(test_doc):
    test_doc = [clean_sentence(text) for text in test_doc]
    new_vecs = [dictionary.doc2bow(test.split()) for test in test_doc]
    new_vec_tfidf_ls = [tfidf[new_vec] for new_vec in new_vecs] 
    sims = [index[new_vec_tfidf] for new_vec_tfidf in new_vec_tfidf_ls]
    sim_index = np.argmax(np.array(sims[0]))
    return sims[1][sim_index]

In [17]:
arr = []
with open("fake.csv",encoding="utf8") as file:
    rows = csv.reader(file)
    for row in rows:
        if(len(row) == 0):
            continue
        if(calRelateRate(row[1:3])):
            arr.append(row)

In [18]:
print(len(arr))

5372


In [19]:
with open("fake_clean.csv","w",encoding="utf8") as file:
    writer = csv.writer(file)
    for row in arr:
        writer.writerow(row)

***

In [4]:
import jieba.posseg as pseg
import csv
def jieba_tokenizer(text):
    words = pseg.cut(text)
    return ''.join([
        word for word, flag in words if flag != 'x'])
def stringify(text):
    return str(text)

In [10]:
all_ = []
with open("./fake_clean2.csv",encoding="utf8") as file:
    rows = csv.reader(file)
    for row in rows:
        if(len(row) == 0):
            continue
        all_.append([jieba_tokenizer(row[0]),jieba_tokenizer(row[1]),row[2]])
print(all_[:10])

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\PC-Henry\AppData\Local\Temp\jieba.cache
Loading model cost 0.841 seconds.
Prefix dict has been built succesfully.


[['看看香港黃之鋒的惡行惡狀欺負老人家算什麼好漢暴徒來台卻變成英雄貴賓把台灣道德價值摧毀殆盡綠色執政崩壞保證', '錯誤網傳影片看看香港黃之鋒的惡行惡狀充分沒有教養的港獨暴力份子...欺負老人家算什麼好漢', 'False'], ['燒烤時戴隱形眼鏡遇高溫會使隱形眼鏡熔化而致盲', '錯誤網傳在烤肉...請將你的隱形眼鏡拿掉一個21歲男生生火時過熱的溫度熔化隱形眼鏡', 'False'], ['剛出爐的麵包傷胃', '謠言終結站剛出爐的麵包會致癌專家斥無稽生活自由時報電子報', 'False'], ['昨晚香港一名參與示威女子被同伴擊中眼睛可能致視力嚴重受損他們竟還反污衊香港警方', '早報中國央視報導香港女子右眼受傷引用假照片新聞信源無可稽考端傳媒InitiumMedia', 'False'], ['你還在吃龍眼嗎看完後震驚了一定要告訴家人啊分享', '假養生近視從800度降到200度謠言讓醫生表示效果有限不要過量了MyGoPen', 'False'], ['你還在吃龍眼嗎看完後震驚了一定要告訴家人啊分享', '龍眼茶降近視謠言說龍眼茶一喝近視度數就狂降這太神奇了我的老天鵝啊', 'False'], ['3C廢材回收換現金', '假圖片3C廢材回收換現金LINE謠言均採用購物抵用金', 'False'], ['直接對口喝易拉罐會致死', '假新聞易開罐飲料對嘴喝會因爲老鼠尿鉤端螺旋體病致死老謠言', 'False'], ['演員張豐毅的孫子被自家養的狗咬出血了得了狂犬病', '假影片被家裡的狗抓到出血小孩發作學狗叫不是狂犬病啦', 'False'], ['嫌犯示範破窗盜竊法一把粗鹽一口口水就可以破窗', '假影片一把粗鹽一口口水就能破窗非鹽巴謠言改編影片', 'False']]


In [12]:
all2_= []
with open("./unfake_clean.csv",encoding="utf8") as file:
    rows = csv.reader(file)
    for row in rows:
        if(len(row[0]) == 0):
            continue
        all2_.append([jieba_tokenizer(row[0]),jieba_tokenizer(row[1]),row[2]])
print(all2_[:10])

[['全台83間夜市200元抵用券下周發放擴大秋冬國旅補助第二波領取方式看過來', '200元夜市抵用券明天發放使用限制一次統整聯合新聞網', 'TRUE'], ['酪梨籽殺死癌細胞食藥署根本不能吃', '聽說酪梨籽可以殺死癌細胞這是真的衛生福利部食品藥物管理署', 'TRUE'], ['恐影響視力嬌生這款隱眼驚見異物漂浮緊急回收500盒LINETODAY', '新頭殼恐影響視力嬌生這款隱眼驚見異物漂浮緊急回收500盒', 'TRUE'], ['恐影響視力嬌生這款隱眼驚見異物漂浮緊急回收500盒LINETODAY', '嬌生急收隱形眼鏡這六款批號產品別再戴熱門話題產業經濟日報', 'TRUE'], ['中橫公路中斷了19年即將在10月底開放通行以後遊客前往梨山就不用繞到合歡山走四五個小時只要70分鐘就能抵達', '東森新聞中橫公路中斷了19年即將在10月底開放通行以後遊客前往梨山就不用繞到合歡山走四五個小時只要70分...Facebook', 'TRUE'], ['發票別傻傻印出來恐錯過百萬獎金LINETODAY', '三立新聞網發票別傻傻印出來恐錯過百萬獎金', 'TRUE'], ['微信偷偷綁定臉書帳號快照這步驟刪除是真的沒那麼恐怖啦...', '微信WeChat自動綁定臉書快刪除事情的真相沒那麼恐怖...蘋果仁你的科技媒體', 'TRUE'], ['微信偷偷綁定臉書帳號快照這步驟刪除是真的沒那麼恐怖啦...', '微信WeChat自動綁定臉書快刪除事情的真相沒那麼恐怖...蘋果仁你的科技媒體', 'TRUE'], ['恐影響視力嬌生這款隱眼驚見異物漂浮緊急回收500盒LINETODAY', '新頭殼恐影響視力嬌生這款隱眼驚見異物漂浮緊急回收500盒', 'TRUE'], ['恐影響視力嬌生這款隱眼驚見異物漂浮緊急回收500盒LINETODAY', '嬌生急收隱形眼鏡這六款批號產品別再戴熱門話題產業經濟日報', 'TRUE']]


# Test re.sub

In [16]:
string = "自由時報黑MyGoPen去到哪?"

In [21]:
import re 
r = "自由時報|MyGoPen|LINETODAY|三立新聞網|政經大事|經濟日報|噓星聞|老外瘋台灣|Google搜尋|Yahoo奇摩新聞|雜誌出版推薦|中時電子報|天下雜誌|ETtoday東森新聞雲|LINE台灣"

In [22]:
re.sub(r,"",string)

'黑去到哪?'

In [1]:
import re 
import pandas as pd

In [6]:
train = pd.read_csv("../fake_clean2.csv",header=None)
train.columns = ["title1","title2","label"]
train.head()

,title1,title2,label
0,看看香港-[黃之鋒]的惡行惡狀欺負老人家算什麼好漢？暴徒來台卻變成英雄貴賓把台灣道德價值摧毀...,【錯誤】網傳影片「看看香港，黃之鋒的惡行惡狀，充分沒有教養的港獨暴力份子，...欺負老人家算...,False
1,3、燒烤時戴隱形眼鏡遇高溫會使隱形眼鏡熔化而致盲,【錯誤】網傳「在烤肉...請將你的隱形眼鏡拿掉！一個21歲男生生火時，過熱的溫度熔化隱形眼鏡」？,False
2,剛出爐的麵包傷胃,《謠言終結站》剛出爐的麵包會致癌？ 專家斥無稽 - 生活 - 自由時報電子報,False
3,昨晚，香港一名參與示威女子被同伴擊中眼睛，可能致視力嚴重受損，他們竟還反污衊香港警方,早報：中國央視報導香港女子右眼受傷，引用假照片，「新聞」信源無可稽考｜端傳媒 Initium...,False
4,你還在吃龍眼嗎？看完後震驚了！一定要告訴家人啊！分享~：,【假養生】近視從800度降到200度？謠言讓醫生表示效果有限，不要過量了！ ｜MyGoPen,False


In [12]:
with open("news_platform.txt") as f:
    news_platform = f.readlines()
news_del = "|".join([re.sub("\W+","",new) for new in news_platform])
with open("other_platform.txt") as f:
    other_platform = f.readlines()
other_del = "|".join([re.sub("\W+","",new) for new in other_platform])
_del = news_del+"|"+other_del

In [15]:
train["title2"] = train["title2"].apply(lambda x:re.sub("\W+","",x)).apply(lambda x:re.sub(_del,"",x))
train["title1"] = train["title1"].apply(lambda x:re.sub("\W+","",x))
train.head()

,title1,title2,label
0,看看香港黃之鋒的惡行惡狀欺負老人家算什麼好漢暴徒來台卻變成英雄貴賓把台灣道德價值摧毀殆盡綠色...,錯誤網傳影片看看香港黃之鋒的惡行惡狀充分沒有教養的港獨暴力份子欺負老人家算什麼好漢,False
1,3燒烤時戴隱形眼鏡遇高溫會使隱形眼鏡熔化而致盲,錯誤網傳在烤肉請將你的隱形眼鏡拿掉一個21歲男生生火時過熱的溫度熔化隱形眼鏡,False
2,剛出爐的麵包傷胃,謠言終結站剛出爐的麵包會致癌專家斥無稽生活,False
3,昨晚香港一名參與示威女子被同伴擊中眼睛可能致視力嚴重受損他們竟還反污衊香港警方,早報中國央視報導香港女子右眼受傷引用假照片新聞信源無可稽考,False
4,你還在吃龍眼嗎看完後震驚了一定要告訴家人啊分享,假養生近視從800度降到200度謠言讓醫生表示效果有限不要過量了,False


# Create Tokenized 建立語料庫 goV+Kaggle+ettoday

In [1]:
import keras
import re
import pandas as pd
import pymongo
import jieba
from langconv import *
jieba.set_dictionary("dict.txt.big.txt")

Using TensorFlow backend.


In [2]:
with open("all_corpus.txt","r",encoding="utf8") as fr:
    all_ls =[f.strip() for f in  fr.readlines()]
print(all_ls[:5])

['太陽能發電比燃煤發電便宜 大陸宣布做到了 | 聯合新聞網', '人家在做、我們在譙！太陽能發電比燃煤發電便宜 大陸宣布做到了！', '新華社於2018/12/29發布新聞稿，中國新聞無從查證起。', '太陽能發電比燃煤發電便宜 大陸宣布做到了 | 聯合新聞網', '人家在做、我們在譙！太陽能發電比燃煤發電便宜 大陸宣布做到了！']


In [3]:
def clean_sentence(sentence):
    r = '\W+'
    sentence = Converter('zh-hant').convert(sentence)
    words = re.sub(r,"",sentence)
    wordarr = [word for word in jieba.cut(words)]
    return " ".join(wordarr)

In [4]:
corpus = pd.DataFrame([clean_sentence(ls) for ls in all_ls])
print(corpus[:5])

Building prefix dict from D:\program\semester_final\clean_data\dict.txt.big.txt ...
Loading model from cache C:\Users\PC-Henry\AppData\Local\Temp\jieba.u6179e01b4c6492e8310058a8757f6034.cache
Loading model cost 1.228 seconds.
Prefix dict has been built succesfully.


                                            0
0         太陽能 發電 比 燃煤 發電 便宜 大陸 宣佈 做到 了 聯合 新聞網
1  人家 在 做 我們 在 譙 太陽能 發電 比 燃煤 發電 便宜 大陸 宣佈 做到 了
2          新華社 於 20181229 發布 新聞稿 中國 新聞 無從查證 起
3         太陽能 發電 比 燃煤 發電 便宜 大陸 宣佈 做到 了 聯合 新聞網
4  人家 在 做 我們 在 譙 太陽能 發電 比 燃煤 發電 便宜 大陸 宣佈 做到 了


In [5]:
K_train =  pd.read_csv("../train_p.csv")
corpus_x1 = K_train.title1_tokenized
corpus_x2 = K_train.title2_tokenized
corpus = pd.concat([
    corpus,corpus_x1, corpus_x2])
print(len(corpus))

718662


In [6]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["semester"]
mycol = mydb["ettoday"]

In [7]:
ettoday_t = []
ettoday_c = []
for i in mycol.find({},{"title":1,"content":1}):
    ettoday_t.append(i["title"].strip())
    ettoday_c.append(i["content"].strip())
ettoday_title = pd.DataFrame([clean_sentence(ls) for ls in ettoday_t])
ettoday_content = pd.DataFrame([clean_sentence(ls) for ls in ettoday_c])
print(ettoday_title[:5])
corpus = pd.concat([
    corpus,ettoday_title, ettoday_content])
print(corpus.shape)

                                        0
0   快訊 高嘉瑜 晚間 好奇 找 阿北 求證 柯 P 終於 一句 話 解答 了
1  美眉 的 夏天 最 HOT 名單 出爐 她 激戰 搶下 冠軍 抱 千元 大獎
2       搭 西寧 軍艦 南偵 22 名 教師 登 太平島 深化 主權 意識
3       桃園 區 慶 讚 中 元 水燈 排車 繞境 傳承 百年 特色 慶典
4       鳳飛飛 故事 展 電影展 等 紀念活動 緬懷 永遠 的 國民 天后
(817988, 1)


In [8]:
tokenizer = keras \
    .preprocessing \
    .text \
    .Tokenizer(200000)

In [9]:
# 去除數字且降維(需要一維)
corpus = corpus[corpus[0].str.isnumeric() == False]
print(corpus.shape)
corpus_a = corpus.values.tolist()
print(corpus_a[:2])
corpus_a = [i[0] for i in corpus_a]

(817983, 1)
[['太陽能 發電 比 燃煤 發電 便宜 大陸 宣佈 做到 了 聯合 新聞網'], ['人家 在 做 我們 在 譙 太陽能 發電 比 燃煤 發電 便宜 大陸 宣佈 做到 了']]


In [10]:
tokenizer.fit_on_texts(corpus_a)
print(tokenizer.num_words)

200000


In [26]:
print(tokenizer.index_word.get(400000))

zone1zone3


In [11]:
import pickle

# 應該將所有資料集整合統一成一個 pickle
with open("tokenized.pickle","wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)